In [1]:
import numpy as np
import qiskit.circuit.library as lib
from qiskit.quantum_info import Operator
from numpy.linalg import multi_dot
from IPython.display import display, Latex

def numpy_to_latex(matrix):
    latex_code = "\\begin{bmatrix}\n"
    for row in matrix:
        latex_code += " & ".join(map(str, row)) + " \\\\\n"
    latex_code += "\\end{bmatrix}"
    return latex_code


Z = Operator(lib.ZGate())
X = Operator(lib.XGate())

def Ry(theta):
    return Operator(lib.RYGate(theta))

def P(theta):
    return Operator(lib.PhaseGate(theta))

u1 = multi_dot([X, Ry(np.arctan(1 + np.sqrt(3))), Z, P(np.pi * 3 / 16), X, P(np.pi * 15 / 16)])
u2 = u1
v1 = multi_dot([X, P(np.pi * 3 / 16), X, P(np.pi * 15 / 16), X, Ry(np.arctan(1 + np.sqrt(3))), Z, X])
v2 = multi_dot([X, Z, X, P(np.pi * 9 / 16), X, P(np.pi * 13 / 16), X, Z, Ry(np.arctan(1 + np.sqrt(3)))])
pi = np.pi
sqrt = np.sqrt
log = np.log

common_log_term = log(-1 / 6 * (-1) ** (1 / 4) * (sqrt(24 - 6 * sqrt(3)) + 1j * (3 + sqrt(3))))
common_factor = 3j + 1j * sqrt(3) + sqrt(6 * (4 - sqrt(3)))
denominator = 15 * pi - 4 * 1j * common_log_term

VCx = np.array([
    [
        -(((-1) ** (5 / 8) * common_factor * (30 * pi - 8 * 1j * common_log_term)) / (24 * denominator)) +
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator),
        0,
        0,
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator) -
        ((-1) ** (5 / 8) * common_factor * (-30 * pi + 8 * 1j * common_log_term)) / (24 * denominator)
    ],
    [
        0,
        -(1 / 2) * (-1) ** (1 / 8) - (1 / 2) * (-1) ** (5 / 8),
        -(1 / 2) * (-1) ** (1 / 8) + (1 / 2) * (-1) ** (5 / 8),
        0
    ],
    [
        0,
        -(1 / 2) * (-1) ** (1 / 8) + (1 / 2) * (-1) ** (5 / 8),
        -(1 / 2) * (-1) ** (1 / 8) - (1 / 2) * (-1) ** (5 / 8),
        0
    ],
    [
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator) -
        ((-1) ** (5 / 8) * common_factor * (-30 * pi + 8 * 1j * common_log_term)) / (24 * denominator),
        0,
        0,
        -(((-1) ** (5 / 8) * common_factor * (30 * pi - 8 * 1j * common_log_term)) / (24 * denominator)) +
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator)
    ]
], dtype=complex)


fCheck = multi_dot([np.kron(u1, u2), VCx, np.kron(v1, v2)]).round(7)

zero_qubit = np.array([[1],[0]])
one_qubit = np.array([[0],[1]])
qubity = [zero_qubit, one_qubit]

In [2]:
fCheck

array([[ 0.5773503-0.j ,  0.5773503-0.j ,  0.5773503-0.j ,
        -0.       -0.j ],
       [ 0.5773503-0.j , -0.2886751+0.5j, -0.2886751-0.5j,
         0.       -0.j ],
       [ 0.5773503-0.j , -0.2886751-0.5j, -0.2886751+0.5j,
         0.       -0.j ],
       [-0.       -0.j ,  0.       -0.j , -0.       -0.j ,
         1.       -0.j ]])

In [3]:
bellstatePhiMin = 1/np.sqrt(2) * (np.kron(qubity[0], qubity[1]) - np.kron(qubity[1], qubity[0]))
bellstatePhiPlus = 1/np.sqrt(2) * (np.kron(qubity[0], qubity[1]) + np.kron(qubity[1], qubity[0]))

In [4]:
wektorwlasny0 = np.array([ 0.888+0.j, -0.46 -0.j,  0.   -0.j])
wektorwlasny1 = np.array([ 0.325+0.j,  0.628+0.j, -0.707+0.j])
wektorwlasny2 = np.array([ 0.325+0.j,  0.628+0.j, 0.707+0.j])

In [5]:
wektorwlasny0

array([ 0.888+0.j, -0.46 +0.j,  0.   +0.j])

In [6]:
bellstatePhiMin

array([[ 0.        ],
       [ 0.70710678],
       [-0.70710678],
       [ 0.        ]])

In [7]:
bellstatePhiPlus

array([[0.        ],
       [0.70710678],
       [0.70710678],
       [0.        ]])

In [65]:
F = fCheck

v0 = np.array([1, 0, 0, 0])  # |00>
v1 = np.array([0, 1, 0, 0])  # |01>
v2 = np.array([0, 0, 1, 0])  # |10>

# Operatory projekcji na stany |v0>, |v1>, |v2>
Pv0 = np.outer(v0, v0.conj())
Pv1 = np.outer(v1, v1.conj())
Pv2 = np.outer(v2, v2.conj())

# Definicja operatora tożsamości I o wymiarach 4x4
I = np.identity(4, dtype=complex)

# Obliczenie F^2 (kwadrat macierzy F)
F_squared = np.linalg.matrix_power(F, 2)

# Konstrukcja bramki CZ o wymiarach 16x16
# CZ = (|v0><v0| ⊗ I) + (|v1><v1| ⊗ F) + (|v2><v2| ⊗ F^2)
CZ12 = np.zeros((16, 16), dtype=complex)

# Dodanie pierwszego członu: |v0><v0| ⊗ I
CZ12 += np.kron(Pv0, I)

# Dodanie drugiego członu: |v1><v1| ⊗ F
CZ12 += np.kron(Pv1, F)

# Dodanie trzeciego członu: |v2><v2| ⊗ F^2
CZ12 += np.kron(Pv2, F_squared)

# Jeśli chcesz uwzględnić czwarty stan |11>, możesz zdefiniować go jako |v3>
v3 = np.array([0, 0, 0, 1])  # |11>
Pv3 = np.outer(v3, v3.conj())

# Możesz zdecydować, jak bramka CZ ma działać na |v3>
# Jeśli chcesz, aby CZ działała jak identyczność na |v3>, dodaj:
CZ12 += np.kron(Pv3, I)

# Opcjonalnie, sprawdź unitarność bramki CZ
is_unitary = np.allclose(CZ.conj().T @ CZ, np.identity(16))


# Teraz możesz zastosować bramkę CZ do dowolnego stanu
# Przykładowo, zastosujmy CZ do stanu |v0> ⊗ |v1>
state = np.kron(v0, v1)

# Zastosowanie bramki CZ
new_state = CZ @ state

# Normalizacja nowego stanu (opcjonalnie)
new_state = new_state / np.linalg.norm(new_state)

# Wyświetlenie wyników


In [71]:
CZ.round(2)

array([[ 1.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ],
       [ 0.  +0.j ,  1.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ],
       [ 0.  +0.j ,  0.  +0.j ,  1.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ],
       [ 0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  1.  +0.j ,  0.  +0.j ,
         0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,
         0.  +0.j ],
       [ 0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.  +0.j ,  0.58+0.j ,
         0.58+0.j ,  0.58+0.j , 

In [60]:
baza = [wektory_wlasne[0], wektory_wlasne[1], wektory_wlasne[2]]
stan0x0 = np.outer(baza[0], baza[0])
stan1x1 = np.outer(baza[1], baza[1])
stan2x2 = np.outer(baza[2], baza[2])

stanPlus = multi_dot([fCheck, baza[0]])

stanPrzygotowany = np.kron(fCheck, np.kron(np.identity(4), np.identity(4)))

Cz01 = np.kron(stan0x0, fCheck) + np.kron(stan1x1, multi_dot([fCheck, fCheck])) + np.kron(stan2x2, np.identity(4))
CZ01 = np.kron(Cz01, np.identity(4))



Cz02 = np.kron(np.kron(stan0x0, np.identity(4)), fCheck) + np.kron(np.kron(stan1x1, np.identity(4)), np.identity(4)) + np.kron(np.kron(stan2x2, np.identity(4)), multi_dot([fCheck, fCheck]))


#print(f'Cz02 = {Cz02}')

Cz12 = np.kron(np.identity(4), np.kron(stan0x0, np.identity(4)) + np.kron(stan1x1, fCheck) + np.kron(stan2x2, multi_dot([fCheck, fCheck])))



U = multi_dot([CZ01, Cz02, stanPrzygotowany])

#print(f'GHZ = {GHZ}')

stanStart = np.kron(baza[0], np.kron(baza[0], baza[0]))

GHZ = multi_dot([U, stanStart])

wspolczynnik_zmiany_bazy = np.kron(np.identity(4), np.kron(fCheck.conj().T, fCheck.conj().T))

GHZqubity = multi_dot([wspolczynnik_zmiany_bazy, GHZ])


In [47]:
np.kron(np.outer(qubity[0], qubity[0]), np.identity(2)) + np.kron(np.outer(qubity[1], qubity[1]), Z)

array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]])

In [48]:
stanPrzygotowany.shape

(64, 64)

In [49]:
GHZ.shape

(64,)

In [50]:
stan0x0.round(3)

array([[ 0.789+0.j, -0.408-0.j,  0.   +0.j,  0.   +0.j],
       [-0.408-0.j,  0.211+0.j, -0.   -0.j,  0.   -0.j],
       [ 0.   +0.j, -0.   -0.j,  0.   +0.j,  0.   +0.j],
       [ 0.   +0.j,  0.   -0.j,  0.   +0.j,  0.   +0.j]])

In [51]:
stan1x1

array([[ 0.10566243+2.90324630e-17j,  0.20412415+2.80432058e-17j,
        -0.22985042-2.25554009e-17j,  0.        +0.00000000e+00j],
       [ 0.20412415+2.80432058e-17j,  0.39433757+0.00000000e+00j,
        -0.44403692+1.74294754e-17j,  0.        +0.00000000e+00j],
       [-0.22985042-2.25554009e-17j, -0.44403692+1.74294754e-17j,
         0.5       -3.92523115e-17j, -0.        +0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        -0.        +0.00000000e+00j,  0.        +0.00000000e+00j]])

In [52]:
stan2x2

array([[0.10566243+2.90324630e-17j, 0.20412415+2.80432058e-17j,
        0.22985042+3.15775612e-17j, 0.        +0.00000000e+00j],
       [0.20412415+2.80432058e-17j, 0.39433757+0.00000000e+00j,
        0.44403692+0.00000000e+00j, 0.        +0.00000000e+00j],
       [0.22985042+3.15775612e-17j, 0.44403692+0.00000000e+00j,
        0.5       +0.00000000e+00j, 0.        +0.00000000e+00j],
       [0.        +0.00000000e+00j, 0.        +0.00000000e+00j,
        0.        +0.00000000e+00j, 0.        +0.00000000e+00j]])

In [53]:
(multi_dot([fCheck, baza[1]]) * np.sqrt(3)).round(3)

array([0.246+0.j   , 0.365+1.156j, 0.365-1.156j, 0.   +0.j   ])

In [54]:
(multi_dot([fCheck, baza[2]]))

array([ 0.95847505+2.57829721e-17j, -0.19772932-3.95718755e-02j,
       -0.19772932+3.95718755e-02j,  0.        +0.00000000e+00j])

In [55]:
Cz01.round(3)

array([[ 0.667+0.j   ,  0.455+0.j   ,  0.455+0.j   ,  0.   +0.j   ,
         0.173+0.j   , -0.236-0.j   , -0.236-0.j   ,  0.   +0.j   ,
        -0.   +0.j   , -0.   -0.j   , -0.   -0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [ 0.455+0.j   , -0.122+0.394j, -0.122-0.394j,  0.   +0.j   ,
        -0.236-0.j   ,  0.322-0.204j,  0.322+0.204j,  0.   +0.j   ,
        -0.   -0.j   ,  0.23 +0.j   , -0.23 -0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [ 0.455+0.j   , -0.122-0.394j, -0.122+0.394j,  0.   +0.j   ,
        -0.236-0.j   ,  0.322+0.204j,  0.322-0.204j,  0.   +0.j   ,
        -0.   -0.j   , -0.23 -0.j   ,  0.23 +0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  1.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ,  0.   +

In [56]:
np.dot(Cz01, Cz01.conj().T).round(2)

array([[ 1.+0.j,  0.-0.j,  0.-0.j,  0.+0.j,  0.+0.j,  0.-0.j,  0.-0.j,
         0.+0.j, -0.-0.j, -0.-0.j, -0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.-0.j, -0.-0.j,  0.+0.j,  0.-0.j,  0.+0.j,  0.+0.j,
         0.+0.j, -0.-0.j,  0.+0.j,  0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j, -0.-0.j,  1.-0.j,  0.+0.j,  0.-0.j,  0.+0.j,  0.+0.j,
         0.+0.j, -0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.-0.j,  0.-0.j,  0.-0.j,
         0.+0.j, -0.-0.j, -0.-0.j, -0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.-0.j,  0.-0.j,  0.+0.j,  0.-0.j,  1.-0.j, -0.-0.j,
         0.+0.j, -0.-0.j,  0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0

In [57]:
(GHZ * np.sqrt(3)).round(3)

array([ 0.424-0.118j, -0.016+0.103j, -0.245+0.015j,  0.   +0.j   ,
       -0.226+0.029j,  0.005+0.094j,  0.056+0.078j,  0.   +0.j   ,
       -0.035+0.09j , -0.052+0.04j , -0.091+0.024j,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
        0.837-0.253j, -0.008+0.119j, -0.404-0.018j,  0.   +0.j   ,
       -0.413-0.025j, -0.003-0.049j,  0.245-0.035j,  0.   +0.j   ,
        0.002+0.126j,  0.036-0.11j ,  0.047-0.012j,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
        0.882+0.314j,  0.015-0.153j, -0.471-0.009j,  0.   +0.j   ,
       -0.471-0.009j, -0.   -0.j   ,  0.244+0.005j,  0.   +0.j   ,
        0.015-0.153j, -0.008+0.079j, -0.   -0.j   ,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j

In [43]:
(GHZqubity).round(3)

array([-0.035+0.068j,  0.084-0.07j ,  0.044+0.002j,  0.   +0.j   ,
        0.072-0.036j,  0.134-0.024j,  0.084+0.054j,  0.   +0.j   ,
        0.056-0.032j,  0.146-0.122j,  0.148-0.044j,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
        0.065-0.049j,  0.095-0.042j,  0.086+0.004j,  0.   +0.j   ,
        0.071+0.023j,  0.294-0.095j,  0.161+0.104j,  0.   +0.j   ,
        0.109-0.062j,  0.282-0.236j,  0.286-0.085j,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
        0.039+0.014j,  0.091-0.002j,  0.115+0.076j,  0.   +0.j   ,
        0.115+0.076j,  0.303+0.108j,  0.295+0.287j,  0.   +0.j   ,
        0.091-0.002j,  0.174-0.12j ,  0.303+0.108j,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,
        0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j

In [44]:
np.kron(baza[0], np.kron(baza[0], baza[0])) + np.kron(baza[1], np.kron(baza[1], baza[1])) + np.kron(baza[2], np.kron(baza[2], baza[2])).round(2)

array([ 0.76474813+1.41558334e-17j, -0.22620252-3.15775612e-17j,
       -0.00471462-1.75963298e-17j,  0.        +0.00000000e+00j,
       -0.22620252-3.15775612e-17j,  0.4458545 +6.91760380e-17j,
       -0.00433757-1.41639579e-17j,  0.        +0.00000000e+00j,
       -0.00471462-1.75963298e-17j, -0.00433757-1.41639579e-17j,
        0.32252879+9.56944614e-18j,  0.        +0.00000000e+00j,
        0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
       -0.22620252-3.15775612e-17j,  0.4458545 +6.91760380e-17j,
       -0.00433757-1.41639579e-17j,  0.        +0.00000000e+00j,
        0.4458545 +6.91760380e-17j,  0.40048319-4.00387432e-17j,
        0.00116123+1.09450662e-17j,  0.        +0.00000000e+00j,
       -0.00433757-1.41639579e-17j,  0.00116123+1.09450662e-17j,
        0.62398152-2.46490005e-17j,  0.        +0.00000000e+00j,
        0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        0.        +0.0000

In [20]:
zeroQ3 = np.array([[1], [0], [0]])
oneQ3 = np.array([[0], [1], [0]])
twoQ3 = np.array([[0], [0], [1]])

In [33]:
zeroXzeroQ3 = np.outer(zeroQ3, zeroQ3)
oneXoneQ3 = np.outer(oneQ3, oneQ3)
twoXtwoQ3 = np.outer(twoQ3, twoQ3)

In [27]:
1j

1j

In [30]:
F = np.array([[1, 0, 0], [0, np.exp(np.pi*2/3 * 1j), 0], [0, 0, np.exp(np.pi*4/3 * 1j)]])

In [32]:
F.round(3)

array([[ 1. +0.j   ,  0. +0.j   ,  0. +0.j   ],
       [ 0. +0.j   , -0.5+0.866j,  0. +0.j   ],
       [ 0. +0.j   ,  0. +0.j   , -0.5-0.866j]])

In [35]:
czTEST = np.kron(zeroXzeroQ3, np.identity(3)) + np.kron(oneXoneQ3, F) + np.kron(twoXtwoQ3, multi_dot([F, F]))

In [36]:
czTEST

array([[ 1. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  1. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  1. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         1. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       , -0.5+0.8660254j,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       , -0.5-0.86

In [37]:
np.dot(czTEST, czTEST.conj().T)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        1.+0.j]])